# LlamaIndex と AgentCore Memory - 学術研究アシスタント（短期メモリ）

## はじめに

このノートブックでは、Amazon Bedrock AgentCore Memory 機能を LlamaIndex と統合して学術研究アシスタントを作成する方法を紹介します。単一の研究セッション内での**短期メモリ**の永続化に焦点を当て、会話全体を通じてアシスタントが論文、発見、研究コンテキストを記憶できるようにします。

## チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントのユースケース | 学術研究アシスタント                                                            |
| エージェントフレームワーク | LlamaIndex                                                                      |
| LLM モデル          | Anthropic Claude 3.7 Sonnet                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、LlamaIndex エージェント、研究ツール                        |
| 難易度              | 初級                                                                            |

学習内容：
- 研究データ永続化のための AgentCore Memory の作成
- LlamaIndex ネイティブメモリ統合の使用
- 論文分析のための研究専用ツールの構築
- 単一セッション内での研究コンテキストの維持
- メモリ境界とセッション分離のテスト

## シナリオのコンテキスト

この例では、研究者が単一の研究セッション内で論文、発見、研究トピックを追跡するのを支援する「学術研究アシスタント」を作成します。アシスタントは AgentCore Memory を使用して、会話全体を通じてレビューした論文、発見した主要な知見、研究の進捗状況に関するコンテキストを維持します。

## アーキテクチャの概要

![LlamaIndex AgentCore 短期メモリアーキテクチャ](LlamaIndex-AgentCore-STM-Arch.png)

## 前提条件

- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory 権限を持つ AWS IAM ロール：
  - `bedrock-agentcore:CreateMemory`
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents`
  - `bedrock-agentcore:RetrieveMemories`
- Amazon Bedrock モデルへのアクセス

## ステップ 1: 依存関係のインストールとセットアップ

In [ ]:
# Install necessary libraries
%pip install llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse boto3

In [ ]:
# Import required components
from bedrock_agentcore.memory import MemoryClient
from llama_index.memory.bedrock_agentcore import AgentCoreMemory, AgentCoreMemoryContext
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from datetime import datetime
import os

## ステップ 2: AgentCore Memory の設定

研究アシスタント用の AgentCore Memory リソースを作成または取得します：

In [ ]:
# Create AgentCore Memory resource
region = os.getenv('AWS_REGION', 'us-east-1')
client = MemoryClient(region_name=region)

try:
    response = client.create_memory_and_wait(
        name=f'AcademicResearchShortTerm_{int(datetime.now().timestamp())}',
        description='Academic research assistant short-term memory for single session context',
        strategies=[],
        event_expiry_days=7,
        max_wait=300,
        poll_interval=10
    )
    memory_id = response['id']
    print(f"✅ Created AgentCore Memory: {memory_id}")
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    memory_id = "your-memory-id-here"  # Replace with existing memory ID

## ステップ 3: 研究ツールの実装

学術研究タスク用の専門ツールを定義します：

In [ ]:
def save_paper_summary(title: str, authors: str, key_findings: str) -> str:
    """Save a research paper summary with title, authors, and key findings"""
    print(f"📄 Saved paper: {title} by {authors}")
    return f"Successfully saved paper summary for '{title}'"

def track_research_topic(topic: str, status: str) -> str:
    """Track research topic progress with current status"""
    print(f"🔬 Tracking research topic: {topic} (Status: {status})")
    return f"Now tracking research topic: {topic} with status {status}"

def save_research_finding(finding: str, confidence: str) -> str:
    """Save a research finding with confidence level"""
    print(f"💡 Research finding saved with {confidence} confidence")
    return f"Saved research finding with {confidence} confidence level"

# Create tool objects for the agent
research_tools = [
    FunctionTool.from_defaults(fn=save_paper_summary),
    FunctionTool.from_defaults(fn=track_research_topic),
    FunctionTool.from_defaults(fn=save_research_finding)
]

## ステップ 4: LlamaIndex エージェントの実装

短期メモリコンテキストを持つ研究アシスタントエージェントを作成します：

In [ ]:
# Configuration for SHORT-TERM memory (single session)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Create memory context for single session
context = AgentCoreMemoryContext(
    actor_id="academic-researcher",
    memory_id=memory_id,
    session_id="research-session-today",  # Same session throughout
    namespace="/academic-research"
)

# Initialize AgentCore Memory and LLM
agentcore_memory = AgentCoreMemory(context=context)
llm = BedrockConverse(model=MODEL_ID)

# Create the research assistant agent
research_agent = FunctionAgent(
    tools=research_tools,
    llm=llm,
    verbose=True
)

print("✅ Academic Research Assistant with short-term memory is ready!")

## ステップ 5: 短期メモリ機能のテスト

包括的な研究セッションを通じて研究アシスタントの短期メモリをテストしましょう。

### テスト 1: セッションの初期化

In [ ]:
# Initialize research session with detailed context
response = await research_agent.run(
    "I'm Dr. Sarah Smith from MIT's Computer Science Department, starting research on 'Machine Learning in Healthcare Applications'. "
    "Track this topic with status 'Literature Review'.",
    memory=agentcore_memory
)

print("🎯 Session Initialization:")
print(response)

### テスト 2: 研究論文の追加

In [ ]:
# Add first paper with detailed metrics
response = await research_agent.run(
    "Save paper: 'Deep Learning for Medical Image Analysis' by Zhang et al. "
    "Key findings: CNNs achieve 95.2% accuracy in chest X-ray diagnosis, 12% improvement over radiologists, "
    "trained on 100,000 images with 0.03 false positive rate.",
    memory=agentcore_memory
)

print("📄 Paper 1 Added:")
print(response)

In [ ]:
# Add second paper with contrasting findings
response = await research_agent.run(
    "Save paper: 'Transformers in Medical NLP' by Johnson et al. "
    "Key findings: BERT models achieve 89.1% F1-score in clinical note classification, "
    "struggle with rare diseases (<70% accuracy), excel at symptom extraction (94% precision).",
    memory=agentcore_memory
)

print("📄 Paper 2 Added:")
print(response)

### テスト 3: アイデンティティとコンテキストの想起

In [ ]:
# Test identity and research context recall
response = await research_agent.run(
    "What's my name, institution, and current research focus?",
    memory=agentcore_memory
)

print("🧠 Identity Recall Test:")
print(response)
print("\n✅ Expected: Dr. Sarah Smith, MIT, Machine Learning in Healthcare")

### テスト 4: 詳細メトリクスの想起

In [ ]:
# Test specific metric recall
response = await research_agent.run(
    "What were the exact accuracy percentages mentioned in the papers I reviewed? "
    "Which authors wrote about CNNs vs Transformers?",
    memory=agentcore_memory
)

print("📊 Detailed Metrics Recall:")
print(response)
print("\n✅ Expected: Zhang et al - CNNs 95.2%, Johnson et al - BERT 89.1%")

### テスト 5: 文脈的推論

In [ ]:
# Test contextual understanding and reasoning
response = await research_agent.run(
    "Based on the papers I've reviewed, which approach would be better for analyzing "
    "chest X-rays vs clinical notes? Explain your reasoning.",
    memory=agentcore_memory
)

print("🤔 Contextual Reasoning Test:")
print(response)
print("\n✅ Expected: CNNs for X-rays (Zhang paper), Transformers for clinical notes (Johnson paper)")

### テスト 6: 研究発見の統合

In [ ]:
# Add synthesized research finding
response = await research_agent.run(
    "Based on Zhang's CNN results (95.2% accuracy) and Johnson's Transformer results (89.1% F1-score), "
    "I conclude that deep learning models consistently achieve >85% accuracy in healthcare tasks. "
    "This finding has high confidence. Save it.",
    memory=agentcore_memory
)

print("🔬 Research Finding Synthesis:")
print(response)

### テスト 7: 相互参照機能

In [ ]:
# Test cross-referencing between findings and papers
response = await research_agent.run(
    "How does my research finding about >85% accuracy relate to the specific results "
    "from Zhang and Johnson? What evidence supports this conclusion?",
    memory=agentcore_memory
)

print("🔗 Cross-Reference Test:")
print(response)
print("\n✅ Expected: Reference to Zhang 95.2% and Johnson 89.1% as supporting evidence")

### テスト 8: 実践的な応用シナリオ

In [ ]:
# Test practical application of accumulated knowledge
response = await research_agent.run(
    "I'm writing a grant proposal for healthcare AI research. What evidence can I cite "
    "about deep learning effectiveness? Include specific numbers and authors.",
    memory=agentcore_memory
)

print("📝 Grant Proposal Support:")
print(response)
print("\n✅ Expected: Comprehensive summary with Zhang 95.2%, Johnson 89.1%, synthesis finding")

## ステップ 6: セッション境界のテスト

異なるセッションを作成して短期メモリの境界をテストしましょう：

In [ ]:
# Create a different session context
new_session_context = AgentCoreMemoryContext(
    actor_id="academic-researcher",
    memory_id=memory_id,
    session_id="different-research-session",  # Different session ID
    namespace="/academic-research"
)

new_session_memory = AgentCoreMemory(context=new_session_context)

# Test memory isolation
response = await research_agent.run(
    "What research have I been working on? What specific accuracy numbers did I find?",
    memory=new_session_memory
)

print("🚧 Session Boundary Test (Different Session):")
print(response)
print("\n✅ Expected: Limited or no recall from previous session (short-term memory boundary)")

In [ ]:
# Return to original session to verify persistence
response = await research_agent.run(
    "Now back in my original session - what were the accuracy numbers from Zhang and Johnson again?",
    memory=agentcore_memory  # Original session memory
)

print("🔄 Original Session Return:")
print(response)
print("\n✅ Expected: Full recall of Zhang 95.2%, Johnson 89.1%")

## 自動テスト検証
これらのセルを実行して、メモリ統合が正しく機能していることを検証します：

In [ ]:
# Define validation functions inline
class TestValidator:
    def __init__(self):
        self.results = {}
    
    def validate_memory_recall(self, response):
        """Check if agent can recall information from earlier in the session"""
        # Check for substantive response (not just "I don't know")
        has_content = len(response) > 50
        # Check for memory indicators
        has_memory_indicators = any(word in response.lower() for word in 
            ['earlier', 'mentioned', 'discussed', 'previously', 'you', 'we', 'our'])
        return "✅ PASS" if (has_content and has_memory_indicators) else "❌ FAIL"
    
    def validate_session_memory(self, response):
        """Check if agent maintains context within session"""
        has_memory_content = len(response) > 100 and any(word in response.lower() for word in 
            ['previous', 'earlier', 'mentioned', 'discussed', 'before', 'already'])
        return "✅ PASS" if has_memory_content else "❌ FAIL"
    
    def validate_cross_reference(self, response):
        """Check if agent can connect current query to previous context"""
        # Look for connecting language
        connecting_words = ['relate', 'connection', 'previous', 'earlier', 'discussed', 
                           'mentioned', 'context', 'based on', 'as we', 'as i']
        has_connection = any(word in response.lower() for word in connecting_words)
        has_substance = len(response) > 80
        return "✅ PASS" if (has_connection and has_substance) else "❌ FAIL"
    
    def run_validation_summary(self, test_results):
        print("🧪 COMPREHENSIVE TEST VALIDATION SUMMARY")
        print("=" * 60)
        
        total_tests = len(test_results)
        passed_tests = sum(1 for result in test_results.values() if "PASS" in result)
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        for test_name, result in test_results.items():
            print(f"{test_name}: {result}")
        
        print("=" * 60)
        print(f"📊 Overall Pass Rate: {passed_tests}/{total_tests} ({pass_rate:.1f}%)")
        
        if pass_rate >= 80:
            print("✅ EXCELLENT: Memory integration working correctly!")
        elif pass_rate >= 60:
            print("⚠️  GOOD: Most memory features working, some issues to investigate")
        else:
            print("❌ NEEDS ATTENTION: Memory integration has significant issues")
        
        return pass_rate

validator = TestValidator()
print("✅ Validation functions loaded!")

In [ ]:
# Run all validation tests
test_results = {}

# Test 1: Memory recall - can the agent recall what was discussed?
response1 = await research_agent.run("What have we discussed so far in this session?", memory=agentcore_memory)
test_results['Memory Recall'] = validator.validate_memory_recall(str(response1))
print(f"レスポンス 1 の長さ: {len(str(response1))} 文字")

# Test 2: Session memory - does the agent maintain context?
response2 = await research_agent.run("What did we talk about earlier?", memory=agentcore_memory)
test_results['Session Memory'] = validator.validate_session_memory(str(response2))
print(f"レスポンス 2 の長さ: {len(str(response2))} 文字")

# Test 3: Cross-reference capability - can it connect to previous context?
response3 = await research_agent.run("How does this relate to what we discussed before?", memory=agentcore_memory)
test_results['Cross Reference'] = validator.validate_cross_reference(str(response3))
print(f"レスポンス 3 の長さ: {len(str(response3))} 文字")

# Display results
validator.run_validation_summary(test_results)

## まとめ

このノートブックでは、以下を実演しました：

- **短期メモリ統合**: セッションスコープの永続化のために AgentCore Memory を LlamaIndex と使用

- **研究専用ツール**: 論文要約、トピック追跡、発見の保存

- **文脈的な会話**: セッション内でアシスタントが詳細情報を記憶

- **相互参照機能**: 複数の論文やインタラクション間で発見を接続

- **セッション境界**: 異なる会話セッション間でのメモリ分離

- **実践的な応用**: 研究助成金申請のサポートと研究の統合

学術研究アシスタントは、短期メモリが単一の研究セッション内で自然で文脈に沿った会話を可能にしながら、異なる会話スレッド間で明確な境界を維持する方法を示しています。

## クリーンアップ

このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう：

In [ ]:
# Clean up AgentCore Memory resource
try:
    client.delete_memory(memory_id)
    print(f"✅ Successfully deleted memory: {memory_id}")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")